### Install Dependencies

In [1]:
!pip install -q jax[cuda12] flax transformers torch datasets accelerate bitsandbytes orbax-checkpoint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 67.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/yashm246/LLM_Response_Time_Optimizer.git

Cloning into 'LLM_Response_Time_Optimizer'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 90 (delta 36), reused 72 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 56.07 KiB | 11.21 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [4]:
%cd LLM_Response_Time_Optimizer/

/content/LLM_Response_Time_Optimizer


In [5]:
import os
print("Current directory:", os.getcwd())
print("\nFiles in src/:")
!ls -la src/

Current directory: /content/LLM_Response_Time_Optimizer

Files in src/:
total 24
drwxr-xr-x 2 root root 4096 Nov  2 00:02 .
drwxr-xr-x 6 root root 4096 Nov  2 00:02 ..
-rw-r--r-- 1 root root   79 Nov  2 00:02 __init__.py
-rw-r--r-- 1 root root 7380 Nov  2 00:02 model_conversion.py
-rw-r--r-- 1 root root 4009 Nov  2 00:02 quantization.py


### Test Conversion with Mistral 7B

In [6]:
import sys
sys.path.append('/content')

In [7]:
from src.model_conversion import convert_model
import time

In [8]:
import sys
sys.path.append('/content')

from src.model_conversion import convert_model
import time

print("=" * 70)
print("Testing Mistral-7B Conversion on Colab")
print("=" * 70)

# Run conversion
start_time = time.time()

try:
    model, params, tokenizer = convert_model(
        model_name="mistralai/Mistral-7B-Instruct-v0.2",
        use_small_model=False  # Use full Mistral
    )

    conversion_time = time.time() - start_time

    print(f"\n✓ Conversion completed in {conversion_time:.2f} seconds")
    print(f"✓ Tokenizer vocab size: {len(tokenizer)}")
    print(f"✓ Top-level param keys: {list(params.keys())}")

    # Inspect structure
    if 'params' in params:
        inner = params['params']
        print(f"✓ Model components: {list(inner.keys())}")

        # Check model structure
        if 'model' in inner:
            model_params = inner['model']
            print(f"✓ Model structure: {list(model_params.keys())}")

            # Check layers
            if 'layers' in model_params:
                num_layers = len(model_params['layers'])
                print(f"✓ Number of layers: {num_layers}")

                # Inspect first layer
                layer_0 = model_params['layers']['0']
                print(f"✓ Layer 0 components: {list(layer_0.keys())}")

    print("\n" + "=" * 70)
    print("✓ Mistral-7B Conversion Successful!")
    print("=" * 70)

except Exception as e:
    print(f"\n✗ Error: {e}")
    import traceback
    traceback.print_exc()

Testing Mistral-7B Conversion on Colab

Starting PyTorch → JAX/Flax Conversion Pipeline

[1/5] Loading PyTorch model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

    ✓ Loaded 291 parameters

[2/5] Converting to JAX arrays...
  ✓ Transposed model.layers.0.self_attn.q_proj.weight: torch.Size([4096, 4096]) → (4096, 4096)
  ✓ Transposed model.layers.0.self_attn.k_proj.weight: torch.Size([1024, 4096]) → (4096, 1024)
  ✓ Transposed model.layers.0.self_attn.v_proj.weight: torch.Size([1024, 4096]) → (4096, 1024)
  ✓ Transposed model.layers.0.self_attn.o_proj.weight: torch.Size([4096, 4096]) → (4096, 4096)
  ✓ Transposed model.layers.0.mlp.gate_proj.weight: torch.Size([14336, 4096]) → (4096, 14336)
  ✓ Transposed model.layers.0.mlp.up_proj.weight: torch.Size([14336, 4096]) → (4096, 14336)
  ✓ Transposed model.layers.0.mlp.down_proj.weight: torch.Size([4096, 14336]) → (14336, 4096)
  ✓ Transposed model.layers.1.self_attn.q_proj.weight: torch.Size([4096, 4096]) → (4096, 4096)
  ✓ Transposed model.layers.1.self_attn.k_proj.weight: torch.Size([1024, 4096]) → (4096, 1024)
  ✓ Transposed model.layers.1.self_attn.v_proj.weight: torch.Size([1024, 4096]) → (4096

### Import Quantization Functions

In [9]:
from src.quantization import quantize_model_params, calculate_size

### Check Original Model Size

In [10]:
print("Measuring FP16 model size...")
fp16_size = calculate_size(params)
print(f"✓ Mistral-7B FP16 size: {fp16_size / 1e9:.2f} GB")
print(f"  ({fp16_size / 1e6:.2f} MB)")

Measuring FP16 model size...
✓ Mistral-7B FP16 size: 14.48 GB
  (14483.46 MB)


### Quantizing Parameters

In [11]:
print("Quantizing Mistral-7B parameters to INT8...")
print("(This will take 1-2 minutes for 7B parameters)\n")

quantized_params, scales = quantize_model_params(params)

print(f"\n✓ Quantization complete!")
print(f"  Number of quantized parameters: {len(scales)}")

Quantizing Mistral-7B parameters to INT8...
(This will take 1-2 minutes for 7B parameters)

  ✓ Quantized params/lm_head/kernel: (4096, 32000) -> INT8 (scale: 0.000727)
  ✓ Quantized params/model/embed_tokens/embedding: (32000, 4096) -> INT8 (scale: 0.000292)
  ○ Kept FP32 params/model/layers/0/input_layernorm/kernel: (4096,)
  ✓ Quantized params/model/layers/0/mlp/down_proj/kernel: (14336, 4096) -> INT8 (scale: 0.000792)
  ✓ Quantized params/model/layers/0/mlp/gate_proj/kernel: (4096, 14336) -> INT8 (scale: 0.000888)
  ✓ Quantized params/model/layers/0/mlp/up_proj/kernel: (4096, 14336) -> INT8 (scale: 0.000406)
  ○ Kept FP32 params/model/layers/0/post_attention_layernorm/kernel: (4096,)
  ✓ Quantized params/model/layers/0/self_attn/k_proj/kernel: (4096, 1024) -> INT8 (scale: 0.001084)
  ✓ Quantized params/model/layers/0/self_attn/o_proj/kernel: (4096, 4096) -> INT8 (scale: 0.000411)
  ✓ Quantized params/model/layers/0/self_attn/q_proj/kernel: (4096, 4096) -> INT8 (scale: 0.002615)
  ✓

### Measure Quantized Model Size

In [12]:
print("Measuring INT8 model size...")
int8_size = calculate_size(quantized_params)

print(f"\n{'='*70}")
print("QUANTIZATION RESULTS - Mistral-7B")
print(f"{'='*70}")
print(f"FP16 size:  {fp16_size / 1e9:>8.2f} GB  ({fp16_size / 1e6:.2f} MB)")
print(f"INT8 size:  {int8_size / 1e9:>8.2f} GB  ({int8_size / 1e6:.2f} MB)")
print(f"Saved:      {(fp16_size - int8_size) / 1e9:>8.2f} GB  ({(fp16_size - int8_size) / 1e6:.2f} MB)")
print(f"Reduction:  {fp16_size / int8_size:>8.2f}x")
print(f"{'='*70}")

Measuring INT8 model size...

QUANTIZATION RESULTS - Mistral-7B
FP16 size:     14.48 GB  (14483.46 MB)
INT8 size:      7.24 GB  (7242.00 MB)
Saved:          7.24 GB  (7241.47 MB)
Reduction:      2.00x


### Analyze Scale Factors

In [13]:
import numpy as np

scale_values = list(scales.values())
print(f"\nScale factor statistics:")
print(f"  Min scale:    {np.min(scale_values):.6f}")
print(f"  Max scale:    {np.max(scale_values):.6f}")
print(f"  Mean scale:   {np.mean(scale_values):.6f}")
print(f"  Median scale: {np.median(scale_values):.6f}")

# Show a few example scale factors
print(f"\nExample quantized parameters:")
for i, (name, scale) in enumerate(list(scales.items())[:5]):
    print(f"  {i+1}. {name}")
    print(f"     Scale: {scale:.6f}")


Scale factor statistics:
  Min scale:    0.000111
  Max scale:    0.002615
  Mean scale:   0.000684
  Median scale: 0.000585

Example quantized parameters:
  1. params/lm_head/kernel
     Scale: 0.000727
  2. params/model/embed_tokens/embedding
     Scale: 0.000292
  3. params/model/layers/0/mlp/down_proj/kernel
     Scale: 0.000792
  4. params/model/layers/0/mlp/gate_proj/kernel
     Scale: 0.000888
  5. params/model/layers/0/mlp/up_proj/kernel
     Scale: 0.000406
